# Mouse Microprotein Atlas Dictionary Builder

**Pan-tissue Atlas of Novel Differential microprotein Analysis**

> ⚠️ **Demonstration Notebook**: This notebook documents the analytical workflow used to build the PANDA atlas. Original data files are not included due to size constraints. See the README for data availability.

## Overview

The PANDA atlas integrates multiple evidence types for microprotein discovery:

| Evidence Type | Description |
|--------------|-------------|
| **Ribo-seq** | Ribosome profiling from 4 tissues (muscle, heart, fat, liver) |
| **ShortStop** | Machine learning microprotein classification |
| **SEER** | Mass spectrometry proteogenomics |
| **BLASTP** | Homology to Swiss-Prot and TrEMBL |
| **smORF types** | Genomic context (uORF, dORF, intergenic, etc.) |
| **Tau scores** | Tissue specificity across 11 tissues |
| **DESeq2** | Differential expression from exercise studies |
| **DeepLoc** | Subcellular localization predictions |

## Citation

If you use this code, please cite:
- Miller et al. (2025) PANDA: Pan-tissue Atlas of Novel Differential microprotein Analysis
- ShortStop: Miller et al. (2025) BMC Methods. PMID: 40756675

---
## Setup and Imports

In [ ]:
# === IMPORTS ===
import pandas as pd
import pickle
import os
import gzip
import re
from collections import Counter
from datetime import datetime

print("✅ All packages imported successfully")

In [ ]:
# === FILE PATHS (for reference - not executable without data) ===
# These paths show the data organization used in our analysis

# Base directory
BASE_DIR = "/path/to/panda/data"

# FASTA sequence files
FASTA_FILES = {
    "shortstop": f"{BASE_DIR}/reference_databases/rp3_riboseq_shortstop_panda_unique_sequences.fasta",
    "ribo_orf": f"{BASE_DIR}/reference_databases/ribo_seq_from_RiboORF_fasta.fasta",
    "gencode": f"{BASE_DIR}/reference_databases/gencode.vM36.pc_translations.fa.gz",
}

# Tissue-specific Ribo-seq count files
TISSUE_RIBOSEQ_FILES = {
    "muscle": f"{BASE_DIR}/riboseq_counts/muscle/combined_ribo_counts.csv_ENMUSG_sequences.csv",
    "heart": f"{BASE_DIR}/riboseq_counts/heart/combined_ribo_counts.csv_ENMUSG_sequences.csv",
    "fat": f"{BASE_DIR}/riboseq_counts/fat/combined_ribo_counts.csv_ENMUSG_sequences.csv",
    "liver": f"{BASE_DIR}/riboseq_counts/liver/combined_ribo_counts.csv_ENMUSG_sequences.csv",
}

# Annotation files
BLASTP_RESULTS = f"{BASE_DIR}/reference_databases/blastp_panda_sequences_vs_uniprot.csv"
SMORF_TYPES = f"{BASE_DIR}/reference_databases/smorf_types_annotations.csv"
TAU_SCORES = f"{BASE_DIR}/deseq/tau_scores_mgi_symbol_sequences.csv"
DESEQ_RESULTS = f"{BASE_DIR}/deseq/filtered_results_padj_less_than_0.05_with_sequence.csv"
DEEPLOC_RESULTS = f"{BASE_DIR}/intermediate_files/deeploc_signalp6_predictions.csv"

# Filtering parameters
MICROPROTEIN_MAX_LENGTH = 150  # amino acids
MIN_CPM_PER_KB_THRESHOLD = 5.0  # for SAM evidence filtering

print("📁 File paths defined (demonstration only)")

---
## 1. Load Reference Sequences

Load FASTA files containing:
1. **ShortStop candidate sequences** - Novel microprotein predictions
2. **RiboORF sequences** - Ribosome profiling-derived ORFs
3. **GENCODE translations** - Canonical protein sequences for comparison

In [ ]:
def load_fasta(fasta_path, is_gzipped=False):
    """
    Load sequences from a FASTA file.
    
    Parameters:
    -----------
    fasta_path : str
        Path to FASTA file
    is_gzipped : bool
        Whether file is gzipped
    
    Returns:
    --------
    dict : {sequence_id: sequence}
    
    Example output:
    {
        'ENSMUST00000001234': 'MKVLWAALLVTFLAGCQA...',
        'chr1:12345-12456(+)': 'MKFLIVAALCGHNPT...',
        ...
    }
    """
    sequences = {}
    current_id = None
    current_seq = []
    
    opener = gzip.open if is_gzipped else open
    mode = 'rt' if is_gzipped else 'r'
    
    with opener(fasta_path, mode) as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if current_id:
                    sequences[current_id] = ''.join(current_seq)
                current_id = line[1:].split()[0]  # Take first word after >
                current_seq = []
            else:
                current_seq.append(line)
        if current_id:
            sequences[current_id] = ''.join(current_seq)
    
    return sequences

# Example usage (not executable without data):
# shortstop_fasta = load_fasta(FASTA_FILES["shortstop"])
# riboorf_fasta = load_fasta(FASTA_FILES["ribo_orf"])
# gencode_fasta = load_fasta(FASTA_FILES["gencode"], is_gzipped=True)

print("✅ FASTA loading function defined")

---
## 2. Build Sequence Dictionary

Create a unified dictionary keyed by **amino acid sequence**.

This structure enables:
- **Deduplication** of identical sequences from different sources
- **Cross-referencing** between databases
- **Evidence aggregation** for multi-source validation

### Dictionary Structure

```python
sequence_dict = {
    'MKVLWAALLVTFLAG...': {  # amino acid sequence as key
        'gene_ids': ['ENSMUST00000001234', 'chr1:12345-12456(+)'],
        'sources': ['ShortStop', 'Muscle-RiboORF'],
        'shortstop': [{'prediction': 'SAM', 'confidence': 0.95}],
        'status': ['Swiss-Prot'],  # or 'TrEMBL' or empty
        'proteogenomics': ['SEER'],  # if detected by mass spec
        'blastp': [{'uniprot_accession_match': 'P12345', 'evalue': 1e-50}],
        'smorf_type': [{'smorf_type': 'uORF', 'gene_body': 'Atf4'}]
    },
    ...
}
```

In [ ]:
# === Initialize Sequence Dictionary ===

sequence_dict = {}

def add_to_dict(sequence, gene_id, source):
    """
    Add or update a sequence entry in the dictionary.
    
    Parameters:
    -----------
    sequence : str
        Amino acid sequence (stop codons will be removed)
    gene_id : str
        Gene/transcript identifier
    source : str
        Data source (e.g., 'ShortStop', 'Muscle-RiboORF', 'GENCODE')
    """
    # Clean sequence - remove stop codons
    clean_seq = sequence.replace('*', '').replace('.', '')
    
    if not clean_seq:
        return
    
    # Initialize entry if new sequence
    if clean_seq not in sequence_dict:
        sequence_dict[clean_seq] = {
            'gene_ids': [],
            'sources': [],
            'shortstop': [],
            'status': [],
            'proteogenomics': [],
            'blastp': [],
            'smorf_type': []
        }
    
    # Add gene_id if not already present
    if gene_id not in sequence_dict[clean_seq]['gene_ids']:
        sequence_dict[clean_seq]['gene_ids'].append(gene_id)
    
    # Add source if not already present
    if source not in sequence_dict[clean_seq]['sources']:
        sequence_dict[clean_seq]['sources'].append(source)

print("✅ Dictionary structure initialized")

# Example usage:
# for gene_id, seq in shortstop_fasta.items():
#     add_to_dict(seq, gene_id, 'ShortStop')
# 
# for gene_id, seq in riboorf_fasta.items():
#     # Determine tissue from gene_id
#     for tissue in ['Muscle', 'Heart', 'Fat', 'Liver']:
#         if tissue.lower() in gene_id.lower():
#             add_to_dict(seq, gene_id, f'{tissue}-RiboORF')
#             break

---
## 3. Integrate BLASTP Annotations

Add homology information from BLASTP searches against UniProt:

| Match Type | Prefix | Interpretation |
|-----------|--------|----------------|
| Swiss-Prot | `sp\|` | Curated canonical protein |
| TrEMBL | `tr\|` | Computationally predicted |
| No match | - | Potentially novel sequence |

In [ ]:
def integrate_blastp(blastp_path, sequence_dict, gene_to_sequence):
    """
    Add BLASTP results to sequence dictionary.
    
    BLASTP was run against UniProt with:
        blastp -query microproteins.fasta -db uniprot_sprot_trembl \
               -outfmt '6 qseqid sseqid pident length evalue bitscore' \
               -evalue 1e-5 -max_target_seqs 1
    
    Parameters:
    -----------
    blastp_path : str
        Path to BLASTP results CSV
    sequence_dict : dict
        Main sequence dictionary
    gene_to_sequence : dict
        Mapping of gene_id -> sequence for lookup
    """
    blastp_df = pd.read_csv(blastp_path)
    print(f"BLASTP results loaded: {len(blastp_df):,} entries")
    
    matched = 0
    for _, row in blastp_df.iterrows():
        gene_id = row.get('gene_id') or row.get('query_id')
        
        if gene_id in gene_to_sequence:
            seq = gene_to_sequence[gene_id]
            
            # Add BLASTP hit details
            blastp_entry = {
                'uniprot_accession_match': row.get('subject_id'),
                'evalue': row.get('evalue'),
                'microprotein_percentage_match': row.get('pident'),
                'blastp_alignment_length': row.get('length'),
                'blastp_bit': row.get('bitscore')
            }
            sequence_dict[seq]['blastp'].append(blastp_entry)
            
            # Determine Swiss-Prot vs TrEMBL status
            subject_id = str(row.get('subject_id', ''))
            if 'sp|' in subject_id:
                if 'Swiss-Prot' not in sequence_dict[seq]['status']:
                    sequence_dict[seq]['status'].append('Swiss-Prot')
            elif 'tr|' in subject_id:
                if 'TrEMBL' not in sequence_dict[seq]['status']:
                    sequence_dict[seq]['status'].append('TrEMBL')
            
            matched += 1
    
    print(f"✅ BLASTP data added: {matched:,} entries matched")

print("✅ BLASTP integration function defined")

---
## 4. Add smORF Type Classifications

Classify each ORF by its genomic context:

| smORF Type | Location | Description |
|------------|----------|-------------|
| **uORF** | 5' UTR | Upstream of main CDS |
| **dORF** | 3' UTR | Downstream of main CDS |

In [ ]:
def integrate_smorf_types(smorf_path, sequence_dict, gene_to_sequence):
    """
    Add smORF type annotations to sequence dictionary.
    
    smORF types were determined by comparing ORF coordinates to 
    GENCODE annotations using bedtools intersect.
    
    Parameters:
    -----------
    smorf_path : str
        Path to smORF types CSV
    sequence_dict : dict
        Main sequence dictionary
    gene_to_sequence : dict
        Mapping of gene_id -> sequence
    """
    smorf_df = pd.read_csv(smorf_path)
    print(f"smORF types loaded: {len(smorf_df):,} entries")
    
    matched = 0
    for _, row in smorf_df.iterrows():
        gene_id = row.get('gene_id')
        if gene_id in gene_to_sequence:
            seq = gene_to_sequence[gene_id]
            smorf_entry = {
                'smorf_type': row.get('smorf_type', 'Unknown'),
                'gene_body': row.get('gene_body', 'Unknown')  # Host gene if applicable
            }
            sequence_dict[seq]['smorf_type'].append(smorf_entry)
            matched += 1
    
    print(f"✅ smORF types added: {matched:,} entries matched")

print("✅ smORF type integration function defined")

---
## 5. Integrate Multi-Tissue Ribo-seq Data

Add ribosome profiling evidence from 4 tissues:

### Normalization Strategy
1. **CPM** (Counts Per Million): Cross-sample normalization
2. **CPM per kb**: Length normalization for fair comparison across ORF sizes

$$\text{CPM} = \frac{\text{raw counts}}{\text{total library counts}} \times 10^6$$

$$\text{CPM per kb} = \frac{\text{CPM} \times 1000}{\text{CDS length (bp)}}$$

In [ ]:
def add_tissue_riboseq_data(tissue_name, file_path, sequence_dict, gene_to_sequence):
    """
    Add tissue-specific Ribo-seq CPM data to sequence dictionary.
    
    Ribo-seq data was generated using:
    1. STAR alignment to mm39 genome
    2. RiboORF for ORF calling
    3. featureCounts for quantification (see featureCounts_riboseq.sh)
    
    Parameters:
    -----------
    tissue_name : str
        Name of tissue (e.g., 'muscle', 'heart')
    file_path : str
        Path to Ribo-seq counts CSV
    sequence_dict : dict
        Main sequence dictionary
    gene_to_sequence : dict
        Gene ID to sequence lookup
    
    Returns:
    --------
    dict : Statistics about the addition
    """
    print(f"\n=== Processing {tissue_name.title()} Ribo-seq ===")
    
    if not os.path.exists(file_path):
        print(f"⚠️  File not found: {file_path}")
        return {'status': 'file_not_found', 'total_added': 0}
    
    tissue_df = pd.read_csv(file_path)
    print(f"Loaded: {len(tissue_df):,} genes")
    
    # Identify count columns (exclude Geneid and sequence)
    exclude_cols = ['Geneid', 'sequence']
    count_cols = [c for c in tissue_df.columns if c not in exclude_cols]
    
    # CPM normalization
    for col in count_cols:
        tissue_df[col] = pd.to_numeric(tissue_df[col], errors='coerce').fillna(0)
    
    total_per_sample = tissue_df[count_cols].sum()
    for col in count_cols:
        tissue_df[f'{col}_cpm'] = (tissue_df[col] / total_per_sample[col]) * 1_000_000
    
    # Calculate mean CPM across replicates
    cpm_cols = [f'{c}_cpm' for c in count_cols]
    tissue_df['mean_cpm'] = tissue_df[cpm_cols].mean(axis=1)
    
    # Initialize tissue field in dictionary
    tissue_field = f'{tissue_name}_riboseq'
    for seq in sequence_dict:
        if tissue_field not in sequence_dict[seq]:
            sequence_dict[seq][tissue_field] = []
    
    # Add data to dictionary
    added = 0
    for _, row in tissue_df.iterrows():
        gene_id = row['Geneid']
        if gene_id in gene_to_sequence:
            seq = gene_to_sequence[gene_id]
            entry = {
                'gene_id': gene_id,
                f'{tissue_name}_riboseq_count': row['mean_cpm']
            }
            sequence_dict[seq][tissue_field].append(entry)
            added += 1
    
    print(f"✅ Added: {added:,} entries")
    return {'status': 'success', 'total_added': added}

print("✅ Ribo-seq integration function defined")

---
## 6. Add Tau Scores (Tissue Specificity)

The **Tau score** measures tissue specificity:

$$\tau = \frac{\sum_{i=1}^{n}(1 - \hat{x}_i)}{n-1}$$

Where $\hat{x}_i = x_i / \max(x)$ is the normalized expression in tissue $i$.

| Tau Value | Interpretation |
|-----------|----------------|
| 0.0 | Ubiquitously expressed |
| 0.5 | Moderate specificity |
| 1.0 | Tissue-specific (single tissue) |

The **Tau driver** is the tissue with maximum expression.

In [ ]:
def integrate_tau_scores(tau_path, sequence_dict):
    """
    Add Tau scores and tissue expression to sequence dictionary.
    
    Tau scores were calculated from bulk RNA-seq across 11 tissues:
    Adrenal, BrownAdip, Heart, Hypothal, Kidney, Liver, Lung,
    Muscle, Pituitary, epididWAT, inguinWAT
    
    Parameters:
    -----------
    tau_path : str
        Path to Tau scores CSV (must have 'sequence' column)
    sequence_dict : dict
        Main sequence dictionary
    """
    tau_df = pd.read_csv(tau_path)
    print(f"Tau scores loaded: {len(tau_df):,} entries")
    print(f"Tau range: {tau_df['Tau'].min():.3f} - {tau_df['Tau'].max():.3f}")
    
    # Tissue expression columns
    tissue_cols = [
        'log2CPM_Adrenal', 'log2CPM_BrownAdip', 'log2CPM_Heart',
        'log2CPM_Hypothal', 'log2CPM_Kidney', 'log2CPM_Liver',
        'log2CPM_Lung', 'log2CPM_Muscle', 'log2CPM_Pituitary',
        'log2CPM_epididWAT', 'log2CPM_inguinWAT'
    ]
    
    matched = 0
    for _, row in tau_df.iterrows():
        seq = row.get('sequence')
        if seq and seq in sequence_dict:
            sequence_dict[seq]['tau_score'] = row['Tau']
            sequence_dict[seq]['tau_driver'] = row['tau_driver']
            
            # Add all tissue expression values
            for col in tissue_cols:
                if col in row:
                    sequence_dict[seq][col] = row[col]
            matched += 1
    
    print(f"✅ Tau scores added: {matched:,} sequences matched")

print("✅ Tau score integration function defined")

---
## 7. Save Sequence Dictionary

Save the complete dictionary as a pickle file for downstream analysis.

In [ ]:
def save_dictionary(sequence_dict, output_dir):
    """
    Save sequence dictionary with timestamp.
    
    Parameters:
    -----------
    sequence_dict : dict
        Complete sequence dictionary
    output_dir : str
        Directory for output files
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Timestamped version for reproducibility
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    timestamped_path = os.path.join(output_dir, f"sequence_dictionary_{timestamp}.pkl")
    
    with open(timestamped_path, 'wb') as f:
        pickle.dump(sequence_dict, f)
    print(f"✅ Saved: {timestamped_path}")
    
    # Latest version for easy loading
    latest_path = os.path.join(output_dir, "sequence_dictionary_latest.pkl")
    with open(latest_path, 'wb') as f:
        pickle.dump(sequence_dict, f)
    print(f"✅ Saved: {latest_path}")
    
    print(f"\n📊 Dictionary contains {len(sequence_dict):,} unique sequences")

print("✅ Save function defined")

---
## 8. Create Microprotein Hits Dictionary

Filter to microproteins (≤150 aa) and create analysis-ready structure with evidence flags.

In [ ]:
def create_hits_dictionary(sequence_dict, max_length=150):
    """
    Create filtered dictionary containing only microproteins with evidence.
    
    Filtering criteria:
    - Length ≤ 150 amino acids (or Swiss-Prot annotated)
    - Has supporting evidence (Ribo-seq, SAM prediction, SEER, or Swiss-Prot)
    
    Parameters:
    -----------
    sequence_dict : dict
        Complete sequence dictionary
    max_length : int
        Maximum microprotein length (default: 150 aa)
    
    Returns:
    --------
    dict : Filtered hits dictionary
    """
    hits_dict = {}
    
    for sequence, data in sequence_dict.items():
        seq_length = len(sequence)
        has_swiss_prot = 'Swiss-Prot' in data['status']
        
        # Filter: microproteins OR Swiss-Prot (any size)
        if seq_length > max_length and not has_swiss_prot:
            continue
        
        # Determine protein class
        protein_class = "Microprotein" if seq_length <= max_length else "Swiss-Prot Protein"
        
        # Determine primary smORF type
        if has_swiss_prot:
            primary_smorf_type = "Swiss-Prot-MP" if seq_length <= 150 else "Swiss-Prot"
        elif 'TrEMBL' in data['status']:
            primary_smorf_type = "TrEMBL"
        elif data.get('smorf_type'):
            types = [e.get('smorf_type', 'Unknown') for e in data['smorf_type']]
            primary_smorf_type = Counter(types).most_common(1)[0][0]
        else:
            primary_smorf_type = "Unknown"
        
        # Evidence flags
        has_sam = any('SAM' in e.get('prediction', '') for e in data['shortstop'])
        has_riboseq = len(data['sources']) > 0
        has_seer = 'SEER' in data['proteogenomics']
        has_trembl = 'TrEMBL' in data['status']
        
        # Store in hits dictionary
        hits_dict[sequence] = {
            'seq_length': seq_length,
            'protein_class': protein_class,
            'gene_count': len(data['gene_ids']),
            'primary_smorf_type': primary_smorf_type,
            'has_sam': has_sam,
            'has_riboseq': has_riboseq,
            'has_seer': has_seer,
            'has_swiss_prot': has_swiss_prot,
            'has_trembl': has_trembl,
            'gene_ids': data['gene_ids'],
            'sources': data['sources'],
            'tau_score': data.get('tau_score'),
            'tau_driver': data.get('tau_driver')
        }
    
    print(f"✅ Created hits dictionary: {len(hits_dict):,} microprotein sequences")
    return hits_dict

print("✅ Hits dictionary function defined")

---
## 9. Export to CSV

Flatten the nested dictionary to a CSV format for analysis in R or other tools.

In [ ]:
def export_to_csv(hits_dict, output_path):
    """
    Convert hits dictionary to flattened CSV.
    
    Parameters:
    -----------
    hits_dict : dict
        Microprotein hits dictionary
    output_path : str
        Path for output CSV
    """
    rows = []
    
    for sequence, data in hits_dict.items():
        row = {
            'sequence': sequence,
            'seq_length': data['seq_length'],
            'protein_class': data['protein_class'],
            'gene_count': data['gene_count'],
            'primary_smorf_type': data['primary_smorf_type'],
            'gene_ids': '; '.join(data['gene_ids']),
            'has_sam': data['has_sam'],
            'has_riboseq': data['has_riboseq'],
            'has_seer': data['has_seer'],
            'has_swiss_prot': data['has_swiss_prot'],
            'has_trembl': data['has_trembl'],
            'tau_score': data['tau_score'],
            'tau_driver': data['tau_driver']
        }
        
        # Simplified status column
        if data['has_swiss_prot']:
            row['status'] = 'Swiss-Prot'
        elif data['has_trembl']:
            row['status'] = 'TrEMBL'
        else:
            row['status'] = 'Noncanonical'
        
        rows.append(row)
    
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False)
    
    print(f"✅ Saved: {output_path}")
    print(f"📊 {len(df):,} sequences with {len(df.columns)} columns")
    
    return df

print("✅ CSV export function defined")

---
## 10. Evidence-Based Filtering

Apply quality filters to retain high-confidence microproteins.

### Filtering Logic

A microprotein is retained if it meets **ANY** of:

1. **Swiss-Prot-MP**: Annotated in Swiss-Prot AND ≤150 aa
2. **SEER**: Detected via proteogenomics (mass spectrometry)
3. **Ribo-seq + valid type**: Has Ribo-seq evidence AND is NOT oORF/isoORF
4. **SAM + CPM threshold**: ShortStop SAM prediction AND CPM/kb > 5 in any tissue

In [ ]:
def apply_evidence_filter(df):
    """
    Apply multi-door evidence filtering.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Microprotein hits dataframe
    
    Returns:
    --------
    pd.DataFrame : Filtered dataframe
    """
    print("=== Applying Evidence-Based Filtering ===")
    
    # Evidence flags
    is_swiss = (df['primary_smorf_type'] == 'Swiss-Prot-MP').fillna(False)
    has_seer = df['has_seer'].fillna(False)
    has_riboseq = df['has_riboseq'].fillna(False)
    has_sam = df['has_sam'].fillna(False)
    
    # Type restriction (exclude oORF, isoORF, Unknown)
    invalid_types = ['oORF', 'isoORF', 'Unknown', '']
    pass_type_check = ~df['primary_smorf_type'].isin(invalid_types)
    
    # CPM threshold (if columns exist)
    cpm_cols = [c for c in df.columns if 'cpm_per_kb' in c.lower()]
    if cpm_cols:
        pass_cpm_check = (df[cpm_cols].fillna(0) > 5).any(axis=1)
    else:
        pass_cpm_check = pd.Series([True] * len(df))
    
    # Apply multi-door filter
    filter_mask = (
        is_swiss |  # Door 1: Swiss-Prot-MP
        has_seer |  # Door 2: SEER proteogenomics
        (has_riboseq & pass_type_check) |  # Door 3: Ribo-seq + valid type
        (has_sam & pass_cpm_check)  # Door 4: SAM + CPM threshold
    )
    
    filtered_df = df[filter_mask].copy()
    
    print(f"Before filtering: {len(df):,}")
    print(f"After filtering: {len(filtered_df):,}")
    print(f"\nEvidence breakdown:")
    print(f"  Swiss-Prot-MP: {is_swiss.sum():,}")
    print(f"  SEER: {has_seer.sum():,}")
    print(f"  Ribo-seq + valid type: {(has_riboseq & pass_type_check).sum():,}")
    print(f"  SAM + CPM: {(has_sam & pass_cpm_check).sum():,}")
    
    return filtered_df

print("✅ Evidence filter function defined")

---
## 11. Generate GTF Tracks

Create subset GTF files for genome browser visualization (UCSC, IGV) and Circos plots.

In [ ]:
def generate_gtf_subsets(panda_df, gtf_path, output_dir):
    """
    Generate GTF subset files for different evidence types.
    
    Creates:
    - RiboSeq_evidence.gtf
    - Swiss-Prot.gtf
    - Mitochondria_predicted.gtf
    - Secreted_predicted.gtf
    
    Parameters:
    -----------
    panda_df : pd.DataFrame
        Filtered PANDA dataframe
    gtf_path : str
        Path to reference GTF
    output_dir : str
        Directory for output GTF files
    """
    # Load reference GTF
    gtf_df = pd.read_csv(
        gtf_path, sep='\t', header=None, comment='#',
        names=['chr', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attr']
    )
    gtf_df['gene_id'] = gtf_df['attr'].str.extract(r'gene_id\s+"([^"]+)"')
    print(f"Reference GTF loaded: {len(gtf_df):,} entries")
    
    os.makedirs(output_dir, exist_ok=True)
    
    def get_gene_ids(df):
        """Extract all gene IDs from semicolon-separated column."""
        ids = set()
        for s in df['gene_ids'].dropna():
            for gid in str(s).split('; '):
                if gid.strip():
                    ids.add(gid.strip())
        return ids
    
    def save_subset(gene_ids, filename, desc):
        subset = gtf_df[gtf_df['gene_id'].isin(gene_ids)]
        if len(subset) > 0:
            output = os.path.join(output_dir, filename)
            subset.to_csv(output, sep='\t', header=False, index=False, quoting=3)
            print(f"✅ {desc}: {len(subset):,} entries -> {filename}")
    
    print("\n=== Generating GTF Subsets ===")
    
    # Ribo-seq evidence
    if 'has_riboseq' in panda_df.columns:
        riboseq_df = panda_df[panda_df['has_riboseq'] == True]
        save_subset(get_gene_ids(riboseq_df), 'RiboSeq_evidence.gtf', 'Ribo-Seq')
    
    # Swiss-Prot
    if 'primary_smorf_type' in panda_df.columns:
        swiss_df = panda_df[panda_df['primary_smorf_type'] == 'Swiss-Prot-MP']
        save_subset(get_gene_ids(swiss_df), 'Swiss-Prot.gtf', 'Swiss-Prot')
    
    # Mitochondria (if DeepLoc data available)
    if 'Localizations' in panda_df.columns:
        mito_df = panda_df[panda_df['Localizations'].str.contains('Mitochondrion', case=False, na=False)]
        save_subset(get_gene_ids(mito_df), 'Mitochondria_predicted.gtf', 'Mitochondria')
    
    # Secreted
    if 'Localizations' in panda_df.columns:
        sec_df = panda_df[panda_df['Localizations'].str.contains('Extracellular', case=False, na=False)]
        save_subset(get_gene_ids(sec_df), 'Secreted_predicted.gtf', 'Secreted')
    
    print(f"\n📁 GTF files saved to: {output_dir}")

print("✅ GTF generation function defined")

---
## Summary

This notebook documented the complete PANDA atlas building workflow:

| Step | Description | Output |
|------|-------------|--------|
| 1 | Load FASTA sequences | `shortstop_fasta`, `riboorf_fasta`, `gencode_fasta` |
| 2 | Build sequence dictionary | `sequence_dict` (nested dict) |
| 3 | Integrate BLASTP | Swiss-Prot/TrEMBL status |
| 4 | Add smORF types | uORF, dORF, etc. |
| 5 | Integrate Ribo-seq | CPM from 4 tissues |
| 6 | Add Tau scores | Tissue specificity |
| 7 | Save dictionary | `sequence_dictionary_latest.pkl` |
| 8 | Create hits dict | Microproteins only |
| 9 | Export CSV | `sequence_hits_dictionary.csv` |
| 10 | Apply filters | Evidence-based filtering |
| 11 | Generate GTFs | Visualization tracks |

### Final Outputs

- `sequence_dictionary_latest.pkl` - Full nested dictionary
- `sequence_hits_dictionary.csv` - Flattened microprotein data  
- `panda_hits_deseq.csv` - With DESeq2 integration
- `panda_hits_deseq_deeploc.csv` - Complete atlas with localization
- `circos_files/*.gtf` - GTF tracks for visualization